In [20]:
#!conda install plotly -y
#!pip install open3d

     |████████████████████████████████| 74.4 MB 450 kB/s eta 0:00:011
     |████████████████████████████████| 10.2 MB 6.8 MB/s eta 0:00:01    |██████████████████▎             | 5.8 MB 5.3 MB/s eta 0:00:01
  Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Using cached ConfigArgParse-1.7-py3-none-any.whl (25 kB)
  Using cached pyquaternion-0.9.9-py3-none-any.whl (14 kB)
     |████████████████████████████████| 75 kB 8.7 MB/s  eta 0:00:01
  Using cached fastjsonschema-2.19.1-py3-none-any.whl (23 kB)
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: nbformat
    Fo

In [1]:
#import cv2
import numpy as np
from matplotlib import pyplot as plt
#import pandas as pd
#import os
#import sys
#import pathlib
import open3d as o3d
import plotly
import plotly.graph_objects as go
#import exifread

In [23]:
#!git clone --recursive https://github.com/mapillary/OpenSfM

Cloning into 'OpenSfM'...
remote: Enumerating objects: 27152, done.
remote: Counting objects: 100% (412/412), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 27152 (delta 156), reused 244 (delta 122), pack-reused 26740
Receiving objects: 100% (27152/27152), 90.21 MiB | 5.64 MiB/s, done.
Resolving deltas: 100% (18695/18695), done.
Submodule 'opensfm/src/third_party/pybind11' (https://github.com/pybind/pybind11.git) registered for path 'opensfm/src/third_party/pybind11'
Cloning into '/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/src/third_party/pybind11'...
remote: Enumerating objects: 27577, done.        
remote: Counting objects: 100% (328/328), done.        
remote: Compressing objects: 100% (149/149), done.        
remote: Total 27577 (delta 196), reused 256 (delta 156), pack-reused 27249        
Receiving objects: 100% (27577/27577), 10.80 MiB | 4.17 MiB/s, done.
Resolving deltas: 100% (19383/19383), done.
Submodule path 'opensfm/src/third_party/pybin

In [4]:
#%cd OpenSfM
!pip install -r requirements.txt

In [5]:
!brew install opencv ceres-solver llvm libomp gcc cmake glog gflags eigen suite-sparse

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
c4core                                   srgn
==> New Casks
arturia-software-center                  wifiman

You have 22 outdated formulae installed.

opencv 4.9.0_4 is already installed but outdated (so it will be upgraded).
To reinstall 2.2.0_1, run:
  brew reinstall ceres-solver
To reinstall 17.0.6_1, run:
  brew reinstall llvm
libomp  is already installed but outdated (so it will be upgraded).
To reinstall 13.2.0, run:
  brew reinstall gcc
To reinstall 3.28.3, run:
  brew reinstall cmake
To reinstall 0.6.0, run:
  brew reinstall glog
To reinstall 2.2.2, run:
  brew reinstall gflags
To reinstall 3.4.0_1, run:
  brew reinstall eigen
To reinstall 7.6.1, run:
  brew reinstall suite-sparse
==> Downloading https://ghcr.io/v2/homebrew/core/opencv/manifests/4.9.0_5
######################################################################### 100.0%
==> Fetching 

In [71]:
!brew list gcc

/usr/local/Cellar/gcc/13.2.0/bin/c++-13
/usr/local/Cellar/gcc/13.2.0/bin/cpp-13
/usr/local/Cellar/gcc/13.2.0/bin/g++-13
/usr/local/Cellar/gcc/13.2.0/bin/gcc-13
/usr/local/Cellar/gcc/13.2.0/bin/gcc-ar-13
/usr/local/Cellar/gcc/13.2.0/bin/gcc-nm-13
/usr/local/Cellar/gcc/13.2.0/bin/gcc-ranlib-13
/usr/local/Cellar/gcc/13.2.0/bin/gcov-13
/usr/local/Cellar/gcc/13.2.0/bin/gcov-dump-13
/usr/local/Cellar/gcc/13.2.0/bin/gcov-tool-13
/usr/local/Cellar/gcc/13.2.0/bin/gfortran
/usr/local/Cellar/gcc/13.2.0/bin/gfortran-13
/usr/local/Cellar/gcc/13.2.0/bin/lto-dump-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-c++-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-g++-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-gcc-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-gcc-ar-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-gcc-nm-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-gcc-ranlib-13
/usr/local/Cellar/gcc/13.2.0/bin/x86_64-apple-darwin23-gf

In [2]:
%cd OpenSfM/

/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM


In [88]:
!brew list ceres-solver

/usr/local/Cellar/ceres-solver/2.2.0_1/include/ceres/ (64 files)
/usr/local/Cellar/ceres-solver/2.2.0_1/lib/libceres.2.2.0.dylib
/usr/local/Cellar/ceres-solver/2.2.0_1/lib/cmake/ (6 files)
/usr/local/Cellar/ceres-solver/2.2.0_1/lib/ (2 other files)
/usr/local/Cellar/ceres-solver/2.2.0_1/share/ceres-solver/ (89 files)


In [6]:
!brew list libomp

/usr/local/Cellar/libomp/18.1.1/include/ (4 files)
/usr/local/Cellar/libomp/18.1.1/lib/libomp.dylib
/usr/local/Cellar/libomp/18.1.1/lib/libomp.a


In [99]:
#!git clone https://ceres-solver.googlesource.com/ceres-solver


Cloning into 'ceres-solver'...
remote: Sending approximately 20.82 MiB ...
remote: Counting objects: 44, done
remote: Total 21851 (delta 13765), reused 21851 (delta 13765)
Receiving objects: 100% (21851/21851), 20.82 MiB | 4.74 MiB/s, done.
Resolving deltas: 100% (13765/13765), done.


In [8]:
#!git submodule update --init --recursive
!python3 setup.py build 1>output.txt 2>&1

In [92]:
!python3 setup.py build_doc
#!python3 -m http.server --directory build/doc/html/

python3: can't open file '/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/src/setup.py': [Errno 2] No such file or directory


In [1]:
!bin/opensfm_run_all data/berlin

zsh:1: no such file or directory: bin/opensfm_run_all


In [35]:
!./viewer/node_modules.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 86829    0 86829    0     0   276k      0 --:--:-- --:--:-- --:--:--  277k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2388    0  2388    0     0   4794      0 --:--:-- --:--:-- --:--:--  4795
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1385    0  1385    0     0   2177      0 --:--:-- --:--:-- --:--:--  2177
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48445    0 48445    0     0   156k      0 --:--:-- --:--:-- --:--:--  156k
  % Total    % Received % Xferd  Average Speed   Tim

In [36]:
!python3 viewer/server.py -d data/berlin

 * Serving Flask app 'server'
 * Debug mode: off
 * Running on all addresses (::)
 * Running on http://[::1]:8080
 * Running on http://[::1]:8080
Press CTRL+C to quit
::1 - - [03/Mar/2024 13:37:24] "GET / HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /node_modules/mapillary-js/dist/mapillary.css HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /styles/opensfm.css HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/opensfm.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/control/CommandExplainerControl.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/control/Copier.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/control/InfoControl.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/control/Popup.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/control/ThumbnailControl.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/controller/DatController.js HTTP/1.1" 200 -
::1 - - [03/Mar/2024 13:37:25] "GET /src/controlle

In [37]:
%cd ..
!ls

/Users/zhang/bsiOpenSFM/Photogrammetry
LICENSE                     camera_mat.npy
OpenSfM                     cmake_build
README.md                   mickey_ears
Structure_from_motion.ipynb notebook_images


In [38]:
# Take initial guess of intrinsic parameters through metadata
!OpenSfM/bin/opensfm extract_metadata mickey_ears/

# Detect features points 
!OpenSfM/bin/opensfm detect_features mickey_ears/

# Match feature points across images
!OpenSfM/bin/opensfm match_features mickey_ears/

# This creates "tracks" for the features. That is to say, if a feature in image 1 is matched with one in image 2,
# and in turn that one is matched with one in image 3, then it links the matches between 1 and 3. In this case, 
# it does not matter since we only have two images
!OpenSfM/bin/opensfm create_tracks mickey_ears/

# Calculates the essential matrix, the camera pose and the reconstructed feature points
!OpenSfM/bin/opensfm reconstruct mickey_ears/

# For visualization using Open3D
!OpenSfM/bin/opensfm export_ply mickey_ears/

Traceback (most recent call last):
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/bin/opensfm_main.py", line 10, in <module>
    from opensfm import commands
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py", line 2, in <module>
    from opensfm import pybundle
ImportError: cannot import name 'pybundle' from partially initialized module 'opensfm' (most likely due to a circular import) (/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py)
Traceback (most recent call last):
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/bin/opensfm_main.py", line 10, in <module>
    from opensfm import commands
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py", line 2, in <module>
    from opensfm import pybundle
ImportError: cannot import name 'pybundle' from partially initialized module 'opensfm' (most likely due to a circular import) (/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py)
Traceback (most re

In [39]:
!OpenSfM/bin/opensfm_run_all mickey_ears/

Traceback (most recent call last):
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/bin/opensfm_main.py", line 10, in <module>
    from opensfm import commands
  File "/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py", line 2, in <module>
    from opensfm import pybundle
ImportError: cannot import name 'pybundle' from partially initialized module 'opensfm' (most likely due to a circular import) (/Users/zhang/bsiOpenSFM/Photogrammetry/OpenSfM/opensfm/__init__.py)


In [50]:
# from PIL import Image

# # Get the image path
# for image_path in pathlib.Path("minecraft_model_final_colorful_bridge/images/").glob("*.png"):
#     # Load the image
#     image = Image.open(image_path)

#     # Remove the EXIF data
#     image.save("minecraft_model_final_colorful_bridge/images/"+image_path.stem+'_without_exif.png', exif=False)

In [5]:
def get_colors_from_ply(plyfile):
    '''
    Function to read the colors from a .ply file
    returns an RGB array scaled between [0,1]
    '''
    with open(plyfile, 'r') as f:
        colors = []
        columns = {}
        col_idx = 0
        header_done = False
        for line in f.readlines():
            if line.startswith('property'):
                columns[line.split()[-1]] = col_idx
                col_idx += 1
            if header_done:
                line_list = line.split()
                colors.append([float(line_list[columns['diffuse_red']]),
                              float(line_list[columns['diffuse_green']]),
                              float(line_list[columns['diffuse_blue']])
                              ])
            if line.startswith('end_header'):
                header_done = True
        colors = np.array(colors)/255
    return colors
        

In [75]:
# read point cloud data
pcd = o3d.io.read_point_cloud("mickey_ears/reconstruction.ply", format='ply')
# store the colors
pcd.colors = o3d.utility.Vector3dVector(get_colors_from_ply('mickey_ears/reconstruction.ply'))

# convert to array
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=5, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.write_html('vis-sparse-mickey_ears.html')


In [76]:
!opensfm undistort mickey_ears/

2023-08-05 03:21:10,854 DEBUG: Undistorting the reconstruction
2023-08-05 03:21:13,348 DEBUG: Undistorting image PXL_20230805_001756557.jpg
2023-08-05 03:21:17,293 DEBUG: Undistorting image PXL_20230805_001757854.jpg
2023-08-05 03:21:21,224 DEBUG: Undistorting image PXL_20230805_001755117.jpg
2023-08-05 03:21:25,154 DEBUG: Undistorting image PXL_20230805_001759151.jpg
2023-08-05 03:21:29,124 DEBUG: Undistorting image PXL_20230805_001800389.jpg
2023-08-05 03:21:33,055 DEBUG: Undistorting image PXL_20230805_001801686.jpg
2023-08-05 03:21:36,994 DEBUG: Undistorting image PXL_20230805_001803763.jpg
2023-08-05 03:21:40,920 DEBUG: Undistorting image PXL_20230805_001804748.jpg
2023-08-05 03:21:45,062 DEBUG: Undistorting image PXL_20230805_001830483.jpg
2023-08-05 03:21:49,175 DEBUG: Undistorting image PXL_20230805_001829375.jpg
2023-08-05 03:21:53,357 DEBUG: Undistorting image PXL_20230805_001831494.jpg
2023-08-05 03:21:57,494 DEBUG: Undistorting image PXL_20230805_001832424.jpg
2023-08-05 03

In [77]:
!opensfm compute_depthmaps mickey_ears/

2023-08-05 03:24:02,411 INFO: Computing neighbors
2023-08-05 03:24:10,124 INFO: Computing depthmap for image PXL_20230805_001756557.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:25:12,346 INFO: Computing depthmap for image PXL_20230805_001757854.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:26:14,388 INFO: Computing depthmap for image PXL_20230805_001755117.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:27:16,696 INFO: Computing depthmap for image PXL_20230805_001759151.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:28:22,638 INFO: Computing depthmap for image PXL_20230805_001800389.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:29:25,409 INFO: Computing depthmap for image PXL_20230805_001801686.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:30:29,452 INFO: Computing depthmap for image PXL_20230805_001803763.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:31:35,066 INFO: Computing depthmap for image PXL_20230805_001804748.jpg with PATCH_MATCH_SAMPLE
2023-08-05 03:32:39,408 INFO: Computing depthmap for image PXL_2023080

In [78]:
pcd_dense = o3d.io.read_point_cloud('mickey_ears/undistorted/depthmaps/merged.ply', format='ply')
pcd_dense.colors = o3d.utility.Vector3dVector(get_colors_from_ply("mickey_ears/undistorted/depthmaps/merged.ply"))
points_dense = np.asarray(pcd_dense.points)
colors_dense = np.asarray(pcd_dense.colors)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points_dense[:,0], y=points_dense[:,1], z=points_dense[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors_dense)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.write_html('vis-dense-mickey_ears.html')

# Additional resources:
This technique is one of the methods used in [photogrammetry](https://en.wikipedia.org/wiki/Photogrammetry), which is used to construct 3D representations of physical objects and locations using photographs and video.

Here are some links to photogrammetry software, tools, and information:

**Free**:
- [OpenDroneMap](https://www.opendronemap.org/)
- [COLMAP](https://colmap.github.io/index.html)
- [Meshroom](https://alicevision.org/#meshroom)
- [MicMac](https://micmac.ensg.eu/index.php/Accueil)
- [OpenMVG](https://github.com/openMVG/openMVG)

**Paid**:
- [Pix4D](https://www.pix4d.com/)
- [3DF Zephyr](https://www.3dflow.net/3df-zephyr-photogrammetry-software/)
- [Agisoft Metascape](https://www.agisoft.com/)
- [Autodesk ReCap](https://www.autodesk.com/products/recap/overview)